###  Created by Luis Alejandro (alejand@umich.edu)

In [1]:
import tensorflow as tf
import numpy as np
from utils import unicode_to_ascii
from dataset import DatasetBuilder
from basic_translation import preprocess
from basic_translation import Translator
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [2]:
# GPU?
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# Creates dataset for training
files = ['../../datasets/nlp/english-spanish.txt']
builder = DatasetBuilder(files, preprocessors=(preprocess,preprocess), batch_size=64, max_obs=40000, test_obs=30)
train_dataset, test_dataset = builder.build()

In [4]:
# Load embedding matrix (glove vectors) -> these will only be used for English (source/input)
embedding_size = 100
word_to_vector = {}
with open('../../datasets/glove.6B/glove.6B.%sd.txt' % embedding_size, encoding='utf8') as file:
    for line in file:
        values = line.split()
        word = unicode_to_ascii(values[0])
        vector = np.asarray(values[1:],dtype=np.float32)
        word_to_vector[word] = vector
        
embedding_vectors = np.zeros((len(builder.source_tokenizer.word_to_index), embedding_size))
for word, index in builder.source_tokenizer.word_to_index.items():
    if word.decode() in word_to_vector:
        embedding_vectors[index,:] = word_to_vector[word.decode()]

In [5]:
# Defining model
translator = Translator(builder.source_tokenizer,
                        builder.target_tokenizer,
                        source_embedding_matrix=embedding_vectors,
                        target_embedding_size=embedding_size,
                        max_output_length=builder.target_tokenizer.max_seq,
                        restore=False,
                        masking=False)

In [6]:
# Training model
translator.train(15, train_dataset)


Epoch 1 out of 15 complete (55.64 secs) -- Train Loss: 1902.9781 -- Train Bleu: 0.01
Epoch 2 out of 15 complete (21.55 secs) -- Train Loss: 1326.9764 -- Train Bleu: 0.06
Epoch 3 out of 15 complete (21.45 secs) -- Train Loss: 1104.8544 -- Train Bleu: 0.10
Epoch 4 out of 15 complete (21.34 secs) -- Train Loss: 940.2594 -- Train Bleu: 0.14
Epoch 5 out of 15 complete (21.48 secs) -- Train Loss: 807.7692 -- Train Bleu: 0.19
Epoch 6 out of 15 complete (21.29 secs) -- Train Loss: 695.4304 -- Train Bleu: 0.24
Epoch 7 out of 15 complete (21.56 secs) -- Train Loss: 599.1246 -- Train Bleu: 0.29
Epoch 8 out of 15 complete (21.68 secs) -- Train Loss: 517.2880 -- Train Bleu: 0.35
Epoch 9 out of 15 complete (21.57 secs) -- Train Loss: 448.2865 -- Train Bleu: 0.40
Epoch 10 out of 15 complete (22.15 secs) -- Train Loss: 390.6614 -- Train Bleu: 0.46
Creating intermediate checkpoint!

Epoch 11 out of 15 complete (22.68 secs) -- Train Loss: 341.9345 -- Train Bleu: 0.51
Epoch 12 out of 15 complete (23.24 

In [7]:
# Test model
translator.evaluate(test_dataset)

Expected: nadie sabe la razon .
Translation: nadie sabe el chiste .

Expected: ¿ tom esta aprendiendo frances ?
Translation: ¿ esta tom frances a frances ?

Expected: pense que tom estaba muerto .
Translation: pense que tom estaba muerto .

Expected: estoy en el toilette .
Translation: estoy en el bano .

Expected: ¿ quienes son tus padres ?
Translation: ¿ quienes son tus amigos ?

Expected: se quebro .
Translation: se acabo todo .

Expected: creo que me gustas .
Translation: me gustas , no te .

Expected: simplemente tiralo .
Translation: simplemente lo intenta .

Expected: esta novela es aburrida .
Translation: este libro es muy pesado .

Expected: la proxima manejo yo .
Translation: de nuevo , viene , por casa .

Expected: cordialmente gracias .
Translation: muchisimas gracias .

Expected: camino junto a ella .
Translation: estoy pintando detras de mary .

Expected: ¿ que tarda tanto ?
Translation: ¿ que tan tan tan largo ?

Expected: tenia un mal dia .
Translation: tuve mucho menos

In [8]:
# Training model
translator.train(15, train_dataset, test_dataset)


Epoch 1 out of 15 complete (22.71 secs) -- Train Loss: 196.0225 -- Train Bleu: 0.69 -- Test Loss: 296.2232 -- Test Bleu: 0.35
Epoch 2 out of 15 complete (22.57 secs) -- Train Loss: 178.6952 -- Train Bleu: 0.71 -- Test Loss: 312.1083 -- Test Bleu: 0.41
Epoch 3 out of 15 complete (22.89 secs) -- Train Loss: 163.1738 -- Train Bleu: 0.73 -- Test Loss: 308.4829 -- Test Bleu: 0.36
Epoch 4 out of 15 complete (22.38 secs) -- Train Loss: 149.8224 -- Train Bleu: 0.75 -- Test Loss: 320.1981 -- Test Bleu: 0.35
Epoch 5 out of 15 complete (22.57 secs) -- Train Loss: 138.1030 -- Train Bleu: 0.77 -- Test Loss: 323.0579 -- Test Bleu: 0.35
Epoch 6 out of 15 complete (23.12 secs) -- Train Loss: 127.7729 -- Train Bleu: 0.79 -- Test Loss: 323.1880 -- Test Bleu: 0.31
Epoch 7 out of 15 complete (23.37 secs) -- Train Loss: 118.7120 -- Train Bleu: 0.80 -- Test Loss: 317.7484 -- Test Bleu: 0.42
Epoch 8 out of 15 complete (22.57 secs) -- Train Loss: 111.2219 -- Train Bleu: 0.81 -- Test Loss: 325.5356 -- Test Bl

In [9]:
# Test model
translator.evaluate(test_dataset)

Expected: nadie sabe la razon .
Translation: nadie sabe la cosa .

Expected: ¿ tom esta aprendiendo frances ?
Translation: ¿ esta tom en frances ?

Expected: pense que tom estaba muerto .
Translation: pense que tom estaba muerto .

Expected: estoy en el toilette .
Translation: estoy en el bano .

Expected: ¿ quienes son tus padres ?
Translation: ¿ quienes son tus amigos ?

Expected: se quebro .
Translation: ha llegado todo .

Expected: creo que me gustas .
Translation: creo que me gusta .

Expected: simplemente tiralo .
Translation: por fin te ha pasado de alli .

Expected: esta novela es aburrida .
Translation: este libro es aburrido .

Expected: la proxima manejo yo .
Translation: de casa , tom va a llegar .

Expected: cordialmente gracias .
Translation: muchisimas gracias .

Expected: camino junto a ella .
Translation: estoy leyendo una vez .

Expected: ¿ que tarda tanto ?
Translation: ¿ que hace tanto calor ?

Expected: tenia un mal dia .
Translation: tuve mucho tiempo .

Expected:

In [10]:
# Creating some input
source = b"I'm very happy to see you."
source = preprocess(tf.constant(source))
source = tf.strings.split(tf.constant(source))
print(source)
source = builder.source_tokenizer.encode(source.numpy())
source = tf.constant(np.pad(source,(0,builder.source_tokenizer.max_seq - len(source))),
                     shape=[1,builder.source_tokenizer.max_seq])
print(source)

tf.Tensor([b'<start>' b'i' b'm' b'very' b'happy' b'to' b'see' b'you' b'.' b'<end>'], shape=(10,), dtype=string)
tf.Tensor([[  1  18  50 947 296 256 105 106   3   4   0   0]], shape=(1, 12), dtype=int32)


In [11]:
# Outputing model translation
prediction = translator.translate(source)
print(' '.join(builder.target_tokenizer.index_to_word[word].decode() for word in prediction))

me alegro que estes feliz . <end>
